In [18]:
import faiss
import numpy as np
import pickle
from PIL import Image

# Load FAISS index
index = faiss.read_index("D:\\project\\cv_final\\embeddings\\faiss_index.bin")

# Load meta.pkl
with open("D:\\project\\cv_final\\embeddings\\meta_clean_original_paths.pkl", "rb") as f:
    meta = pickle.load(f)

# Load image embeddings nếu cần query bằng vector
image_embeds = np.load("D:\\project\\cv_final\\embeddings\\image_embeds_500.npy")

In [19]:
print(meta.keys())

dict_keys(['title', 'ingredients', 'cleaned_ingredients', 'instructions', 'image_path'])


In [ ]:
# sửa lại image_path trong meta
import os
import pickle


meta_path = "D:\\project\\cv_final\\embeddings\\meta_clean_original_paths.pkl"
image_folder = "D:\\project\\cv_final\\data\\Food-Images"

new_paths = []
for path in meta["image_path"]:
    # đảm bảo có .jpg
    name = os.path.basename(path)
    if not name.lower().endswith(".jpg"):
        name = name + ".jpg"

    full_path = os.path.join(image_folder, name)
    new_paths.append(full_path)

# 3. Ghi đè lại meta.pkl
meta["image_path"] = new_paths

with open(meta_path, "wb") as f:
    pickle.dump(meta, f)

print("Đã cập nhật toàn bộ image_path trong meta.pkl")

Đã cập nhật toàn bộ image_path trong meta.pkl


In [36]:
print(image_embeds.shape)


(500, 768)


In [7]:
query_emb = image_embeds[100].reshape(1, -1)
query_emb = query_emb / np.linalg.norm(query_emb, axis=1, keepdims=True)
D, I = index.search(query_emb, k=5)

# 6. Print results
print("Top 5 similar images:")
for dist, idx in zip(D[0], I[0]):
    print(f"Distance: {dist:.4f}")
    print("   Image Path:", meta["image_path"][idx])
    print("   Title:", meta["title"][idx])
    print("   Ingredients:", meta["cleaned_ingredients"][idx])
    print("---")

AssertionError: 

In [20]:
print(meta["image_path"][100])  # Hiển thị đường dẫn ảnh truy vấn

D:\project\cv_final\data\Food-Images\stuffed-eggplants-and-zucchini-tomato-sauce-falastin.jpg


In [21]:
import pandas as pd
df = pd.read_csv(r"D:\project\cv_final\data\cleaned_df.csv")
df.iloc[100]

Unnamed: 0                                                           100
title                  Stuffed Eggplants and Zucchini in a Rich Tomat...
ingredients            ['2½ tbsp olive oil', '2 onions, finely diced ...
cleaned_ingredients    onion in zucchini 10g oil tomato tomatoes gree...
instructions           To make the sauce, put the oil into a saucepan...
image_path             D:\food_recommender\data\Food Images\stuffed-e...
Name: 100, dtype: object

In [ ]:
import os
image_path=meta["image_path"][100]
image_name = os.path.basename(image_path)
print(image_name)

stuffed-eggplants-and-zucchini-tomato-sauce-falastin.jpg


# retieval

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import torch
import faiss
import numpy as np
import pickle



model_name = "openai/clip-vit-base-patch32"

tokenizer = CLIPTokenizer.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

d:\app1\anaconda3\envs\ds\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ThinkBook\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [39]:
MODEL_DIR = "D:\\project\\cv_final\\model\\clip_local"
IMAGE_EMBED_PATH = r"D:\project\cv_final\embeddings\image_embeds_500.npy"
TEXT_EMBED_PATH = r"D:\project\cv_final\embeddings\text_embeddings_cleaned.npy"
META_PATH = r"D:\project\cv_final\embeddings\meta_clean_original_paths.pkl"
TEXT_FAISS_PATH = r"D:\project\cv_final\embeddings\faiss_text_index_cleaned.bin"
IMAGE_FAISS_PATH = r"D:\project\cv_final\embeddings\faiss_image_index_cleaned.bin"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [40]:
index = faiss.read_index(TEXT_FAISS_PATH)

text_embeds = np.load(TEXT_EMBED_PATH)
dim = text_embeds.shape[1]
print("Embedding dim:", dim)

with open(META_PATH, "rb") as f:
    meta = pickle.load(f)

print("Loaded", len(meta['instructions']), "recipes")

Embedding dim: 512
Loaded 13471 recipes


In [41]:
def encode_text(text):
    inputs = tokenizer(text=[text], images=None, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        emb = model.get_text_features(**inputs)
    emb = emb.cpu().numpy()
    return emb / np.linalg.norm(emb)


# ------------------------------
# Search
# ------------------------------
def search_recipes_by_text(query: str, top_k: int = 5):
    query_emb = encode_text(query)

    distances, indices = index.search(query_emb, top_k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        item = {
            "score": float(dist),
            "title": meta["title"][idx],
            "ingredients": meta["ingredients"][idx],
            "instructions": meta["instructions"][idx],
            "image_path": meta["image_path"][idx]
        }
        results.append(item)

    return results

In [48]:
query = "coffee, flour, sugar, butter, eggs"
results = search_recipes_by_text(query, top_k=5)

for r in results:
    print("\n--- Result ---")
    print("Score:", r["score"])
    print("Title:", r["title"])
    print("Image:", r["image_path"])
    print("Ingredients:", r["ingredients"])
    print("Instructions:", r["instructions"][:200], "...")

Keyword arguments {'images': None} not recognized.



--- Result ---
Score: 35.75010299682617
Title: Gardener's Pizza
Image: D:\project\cv_final\data\Food-Images\gardeners-pizza-395569.jpg
Ingredients: ['Generous pinch of sugar', '1 1/4 cups/300 ml warm water {about 110°F/45°C}', 'One 1/4-oz/7-g package active dry yeast', '2 tbsp extra-virgin olive oil, plus more for drizzling', '2 cups/255 g bread/strong flour', '1 to 1 1/4 cups/175 to 200 g whole-wheat/wholemeal or white whole-wheat/wholemeal flour', '1 tsp fine sea salt', 'Cornmeal or semolina flour', '4 to 6 oz/115 to 170 g cheese, such as shredded or fresh mozzarella, burrata, or fresh ricotta', '2 garlic cloves, very thinly sliced', '2 to 4 handfuls fresh veggies such as sliced tomatoes, arugula/rocket, sorrel, shaved zucchini/courgette, kale, Swiss chard, corn, or mushrooms', 'Handful of fresh herbs, such as basil, opal basil', 'Parmigiano-Reggiano cheese', 'Extra-virgin olive oil', 'Fine sea salt', 'Freshly ground black pepper']
Instructions: Make the dough: Whisk a pinch of suga

In [28]:
print("FAISS index size:", index.ntotal)
print("Meta size:", len(meta))

FAISS index size: 13501
Meta size: 5


In [22]:
print("FAISS index dim =", index.d)
q = encode_text("test")
print("Query embedding dim =", q.shape)

FAISS index dim = 512
Query embedding dim = (1, 768)


In [53]:
from PIL import Image
def load_system():
    print("Loading CLIP from local...")
    processor = CLIPProcessor.from_pretrained(MODEL_DIR, local_files_only=True)
    model = CLIPModel.from_pretrained(MODEL_DIR, local_files_only=True).to(DEVICE)

    print("Loading embeddings...")
    image_embeds = np.load(IMAGE_EMBED_PATH)

    print("Loading FAISS index...")
    index = faiss.read_index(IMAGE_FAISS_PATH)

    print("Loading metadata...")
    with open(META_PATH, "rb") as f:
        meta = pickle.load(f)

    return processor, model, image_embeds, index, meta
def encode_image(image_path, processor, model):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        emb = model.get_image_features(**inputs)

    emb = emb.cpu().numpy().astype("float32")
    emb /= np.linalg.norm(emb, axis=1, keepdims=True)
    return emb
def search_image(query_path, top_k=5):
    processor, model, image_embeds, index, meta = load_system()

    print(f"\nEncoding query image: {query_path}")
    query_emb = encode_image(query_path, processor, model)

    print("\nSearching FAISS...")
    distances, indices = index.search(query_emb, top_k)

    print("\nTop-k similar recipes:")
    for dist, idx in zip(distances[0], indices[0]):
        print(f"- {dist:.4f}  |  {meta['title'][idx]}  |  {meta['image_path'][idx]}")



In [55]:
TEST_IMAGE = r"D:\project\cv_final\data\Food-Images\-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg"

search_image(TEST_IMAGE, top_k=5)

Loading CLIP from local...
Loading embeddings...
Loading FAISS index...
Loading metadata...

Encoding query image: D:\project\cv_final\data\Food-Images\-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg

Searching FAISS...

Top-k similar recipes:
- 0.3138  |  Grilled Pizza  |  D:\project\cv_final\data\Food-Images\grilled-pizza-best-dough-toppings-grilling.jpg
- 0.3230  |  Grilled Peach Toast With Pimiento Cheese  |  D:\project\cv_final\data\Food-Images\grilled-peach-toast-with-pimiento-cheese.jpg
- 0.3403  |  Open-Face Mushroom Sandwiches With Pecorino Salsa Verde  |  D:\project\cv_final\data\Food-Images\open-face-mushroom-sandwiches-pecorino-salsa-verde.jpg
- 0.3630  |  Broiled Goat Cheese Toasts With Marinated Greens  |  D:\project\cv_final\data\Food-Images\broiled-goat-cheese-toasts-with-marinated-greens.jpg
- 0.3997  |  Silken Tofu With Soy-Sauced Tomatoes  |  D:\project\cv_final\data\Food-Images\silken-tofu-with-soy-sauced-tomatoes.jpg


In [34]:
print("Query embedding shape:", query_emb.shape)

Query embedding shape: (1, 768)


In [37]:
# Embedding dimension từ file đã load
print("Image embedding file dim:", image_embeds.shape)

# FAISS index dim
print("FAISS index dim:", index.d)

# Embedding query dim
qe = encode_image(TEST_IMAGE, processor, model)
print("Query dim:", qe.shape)


Image embedding file dim: (500, 768)
FAISS index dim: 512
Query dim: (1, 768)
